In [ ]:
def get_data(interval, start = "2017", end = "2023", assets = None):
    data = Get_m_data(interval = "1h", assets=assets)
    if assets is not None:
        data_r = data.pct_change().loc[start : end, assets].dropna()
    else:
        data_r = data.pct_change().loc[start : end].dropna()
    return data_r

In [ ]:
RISKY_ASSETS = ["BTC", "TWT", "QNT"]
START = "2022"
END = "2023"

data_r = get_data(interval="1d", assets = RISKY_ASSETS)
data_r.head()

In [ ]:
data_r.plot(subplots = True, figsize = (15, 8))

In [ ]:
coeffs = []
cond_vol = []
std_resids = []
models = []

for asset in data_r.columns:
    model = arch_model(data_r[asset], mean="Constant", vol="GARCH", p=1, q=1)
    model = model.fit(update_freq=0, disp="off")
    coeffs.append(model.params)
    cond_vol.append(model.conditional_volatility)
    std_resids.append(model.std_resid)
    models.append(model)
    

In [ ]:
coeffs_df = pd.DataFrame(coeffs, index=data_r.columns)

cond_vol_df = (
    pd.DataFrame(cond_vol).transpose().set_axis(data_r.columns, axis="columns")
)

std_resids_df = (
    pd.DataFrame(std_resids).transpose().set_axis(data_r.columns, axis="columns")
)

In [ ]:
R = (
    std_resids_df.transpose().dot(std_resids_df).div(len(std_resids_df))
)
R

In [ ]:
# Define objects

diag = []

D = np.zeros((len(RISKY_ASSETS), len(RISKY_ASSETS)))
D

for model in models:
    diag.append(
        model.forecast(horizon=1).variance.iloc[-1, 0]
    )
diag = np.sqrt(diag)
    
# Fill the diagonal of D with values from diag
np.fill_diagonal(D, diag)
    
# Calculate the conditional covariance matrix
H = np.matmul(np.matmul(D, R.values), D)

H

# DCC-GARCH

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

#install.packages('rmgarch', repos='http://cran.us.r-project.org')
library(rmgarch)

In [ ]:
%R -i data_r

print(data_r)

In [ ]:
# Define the model specification

%R

# define GARCH(1, 1) model

univariate_spec <- ugarchspec(
    mean.model == list(armaOrder = c(0, 0)),
    variance.model == list(garchOrder = c(1, 1),
                          model = "sGARCH"),
    distribution.model == "norm"
)

# define DCC(1, 1) model
n <- dim(data_r)[2]

dcc_spec <- dccspec(
    uspec = multispec(replicate(n, univariate_spec)),
    dccOrder = c(1, 1),
    distribution = "mvnorm"
)



In [ ]:
# Estimate the model

%R

dcc_fit <- dccfit(dcc_spec, data=data_r)
dcc_fit

In [ ]:
# Calculate the five-step ahead forecasts
forecasts <- dccforecast(dcc_fit, n.head = 5)

In [ ]:
%R
# Conditional covariance matrix
forecasts @ mforecast $ H

In [ ]:
%R
# Conditional correlation matrix
forecasts @ mforecast $ R

In [ ]:
%R
# Proxy correlation process
forecasts @ mforecast $ Q

In [ ]:
%R
# Conditional mean forecasts
forecasts @ mforecast $ mu